# Finding cloud products!
Want data from the cloud, but don't know what's available or how to access it? Look no further.

# AWS

#### GOES (16, 17, 18) buckets: https://registry.opendata.aws/noaa-goes/

In [ ]:
import requests
import s3fs
import netCDF4

# Select your desired variables by looking at the product documentation (see URL above!)
satellite = "noaa-goes16"
product = "ABI-L2-LSTM"
year = "2023"
day_of_year = "004"
hour = "17"

name_string = f'{satellite}/{product}/{year}/{day_of_year}/{hour}'

# AWS's cloud code
aws = s3fs.S3FileSystem(anon=True)
data_files = aws.ls(name_string, refresh=True)

# See which files your search produced.
fnames = []

for f in data_files:
    fname = f[len(satellite)+1:]
    fnames.append(fname)
    print(fname)
    
# Crack open the data. You can now use this netCDF file as you would any downloaded netCDF file.
for file in fnames:
    resp = requests.get(f'https://{satellite}.s3.amazonaws.com/{file}')
    nc = netCDF4.Dataset(file, memory = resp.content)

# Azure

#### GOES (16, 17, 18): https://microsoft.github.io/AIforEarthDataSets/data/goes-r.html

In [ ]:
import requests
from azure.storage.blob import ContainerClient
import netCDF4

# Select your desired variables by looking at the product documentation (see URL above!)
satellite = "noaa-goes16"
product = "ABI-L2-LSTM"
year = "2023"
day_of_year = "004"
hour = "17"

name_string = f'{product}/{year}/{day_of_year}/{hour}/'

# Azure's cloud code
goes_container_client = ContainerClient(account_url='https://goeseuwest.blob.core.windows.net', 
                                                     container_name=satellite, credential=None)

# See which files your search produced.
fnames = []
for f in goes_container_client.walk_blobs(name_starts_with=name_string, delimiter='/'):
    fnames.append(f.name)
    print(f.name)
           
# Crack open the data. You can now use this netCDF file as you would any downloaded netCDF file.
for file in fnames:
    resp = requests.get(f'https://goeseuwest.blob.core.windows.net/{satellite}/{file}')
    nc = netCDF4.Dataset(file, memory = resp.content)

# Google

#### GOES (16, 17, 18): https://console.cloud.google.com/marketplace/product/noaa-public/goes

In [ ]:
import requests
from google.cloud import storage
import netCDF4

# Select your desired variables by looking at the product documentation (see URL above!)
satellite = "gcp-public-data-goes-16"
product = "ABI-L2-LSTM"
year = "2023"
day_of_year = "004"
hour = "17"

name_string = f'{product}/{year}/{day_of_year}/{hour}/'

# Google's cloud code
client = storage.Client.create_anonymous_client()
bucket = client.bucket(bucket_name = satellite)

blobs = bucket.list_blobs(prefix = name_string, delimiter = "/")
response = blobs._get_next_page_response()

# See which files your search produced.
fnames = []
urls = []

for f in range(0, len(response["items"])):
    url_link = ((response["items"])[f])["mediaLink"]
    urls.append(url_link)
    fname = ((response["items"])[f])["name"]
    fnames.append(fname)
    
    print(fname)
    
# Crack open the data. You can now use this netCDF file as you would any downloaded netCDF file.
for i in range(0,len(fnames)):
    resp = requests.get(urls[i])
    nc = netCDF4.Dataset(fnames[i], memory = resp.content)

In [ ]:
# Explore Google's available products:
from google.cloud import storage

products = []

client = storage.Client.create_anonymous_client()
bucket = client.bucket(bucket_name = satellite)
blobs = bucket.list_blobs(delimiter = "/")
response = blobs._get_next_page_response()
for product in response['prefixes']:
    print(product)